In [17]:
from openai import OpenAI

Desafio Final:
1. Carregar um arquivo .txt onde cada linha será um elemento de uma lista do Python.
2. Mandá-la ao modelo que você está rodando localmente para extrair, em formato JSON, onde cada item terá "usuário", "resenha_original", "resenha_pt", "sentimento".
3. Transformar a resposta do modelo em uma lista de dicionários Python.
4. Criar uma função que, dada uma lista de dicionários percorre a lista e faz duas coisas:
    * conta a quantidade de avaliações positivas, negativas e neutras;
    * une cada item dessa lista em uma variável do tipo string com algum separador.

Ao final retorna ambas as coisas.

In [18]:
import pandas as pd

In [19]:
df = pd.read_csv("../data/Resenhas_App_ChatGPT.txt", sep="$", header=None)
df.columns = ["id", "usuario", "resenha_original"]
df.head()

,id,usuario,resenha_original
0,53409593,Safoan Riyad,J'aimais bien ChatGPT. Mais la dernière mise à...
1,39485494,Habimana Therese,This app is very important but sometimes it gi...
2,4549594,Shahidatun jannat,Wonderful app..i just aastonished.. love this ...
3,890535,Rayyan R,Anytime i try to get the app to work. It tells...
4,3590353,Nkanyi Cele,Acabo de instalar la aplicación desde Google P...


In [20]:
client = OpenAI(
    base_url = "http://127.0.0.1:1234/v1",
    api_key = "<KEY>"
)

def analisar_sentimento(review):
    try:
        response = client.chat.completions.create(
            model="google/gemma-3-1b",
            messages=[
                {"role": "system", "content": "Você faz exatamente o que pedido de você sem mais nem menos."}, # pode determinar comportamento
                {"role": "user", "content": f"""Você irá analisar a resenha que eu te mandarei abaixo e retorna com uma análise de sentimento.
                                                Você deve responder APENAS com uma das seguintes palavras: positivo, negativo ou neutro, indicando o sentimento relativo àquela resenha específica.

                                                Exemplos:
                                                'Wonderful app..i just aastonished.. love this app..' -> positivo
                                                'Anytime i try to get the app to work. It tells me "an error has occured" can this be fixed' -> negativo
                                                'This app is very important but sometimes it gives lies' -> neutro

                                                Resenha a ser analisada: {review}
                                                """},
            ],
            temperature=0.5,
            max_tokens=300
        )
        return response.choices[0].message.content
    except Exception as e:
        print(e)

In [21]:
def traduzir_review(review):
    try:
        response = client.chat.completions.create(
            model="google/gemma-3-1b",
            messages=[
                {"role": "system", "content": "Você faz exatamente o que pedido de você sem mais nem menos."}, # pode determinar comportamento
                {"role": "user", "content": f"""Você irá analisar a resenha que eu te mandarei abaixo e retorna a tradução dela para o português brasileiro.
                                                Você deve responder APENAS com o texto traduzido e nada mais.

                                                Resenha a ser traduzido: {review}
                                                """},
            ],
            temperature=0.5,
            max_tokens=300
        )
        return response.choices[0].message.content
    except Exception as e:
        print(e)

In [65]:
df["resenha_pt"] = df["resenha_original"].apply(traduzir_review)
df["sentimento"] = df["resenha_original"].apply(analisar_sentimento)
df["sentimento"] = df["sentimento"].str.strip().lower()
df.head()

Connection error.
Connection error.
Connection error.
Connection error.
Connection error.
Connection error.
Connection error.
Connection error.
Connection error.


KeyboardInterrupt: 

In [41]:
df.to_csv("../data/reviews_gpt_desafio.csv")

In [29]:
df_json = df.to_json(orient="records")
df_dict = df.to_dict(orient="records")
df_dict

[{'id': 53409593,
  'usuario': 'Safoan Riyad',
  'resenha_original': "J'aimais bien ChatGPT. Mais la dernière mise à jour a tout gâché. Elle a tout oublié.",
  'resenha_pt': 'Eu gosto muito de ChatGPT. A última atualização não funcionou bem. Ela esqueceu tudo.\n',
  'sentimento': 'negativo'},
 {'id': 39485494,
  'usuario': 'Habimana Therese',
  'resenha_original': 'This app is very important but sometimes it gives lies',
  'resenha_pt': 'Este aplicativo é muito importante, mas às vezes dá mentiras.',
  'sentimento': 'negativo'},
 {'id': 4549594,
  'usuario': 'Shahidatun jannat',
  'resenha_original': 'Wonderful app..i just aastonished.. love this app..',
  'resenha_pt': 'App maravilhoso! Estou impressionado demais... amo este aplicativo!',
  'sentimento': 'positivo'},
 {'id': 890535,
  'usuario': 'Rayyan R',
  'resenha_original': 'Anytime i try to get the app to work. It tells me "an error has occured" can this be fixed',
  'resenha_pt': 'Aviso de erro ocorrido.',
  'sentimento': 'nega

4. Criar uma função que, dada uma lista de dicionários percorre a lista e faz duas coisas:
    * conta a quantidade de avaliações positivas, negativas e neutras;
    * une cada item dessa lista em uma variável do tipo string com algum separador.

In [66]:
def contar_sentimentos(reviews_lista_dict):
    n_positivos, n_negativos, n_neutros = 0, 0, 0
    lista_dict = []

    for item in reviews_lista_dict:
        if item["sentimento"] == "positivo":
            n_positivos = n_positivos + 1
        elif item["sentimento"] == "negativo":
            n_negativos = n_negativos + 1
        else:
            n_neutros = n_neutros + 1

        lista_dict.append(str(item))

    lista_unida = "#####".join(lista_dict)

    return n_positivos, n_negativos, n_neutros, lista_unida


In [67]:
positivos, negativos, neutros, txt = contar_sentimentos(df_dict)
print(positivos, negativos, neutros)

5 14 5


In [39]:
df_dict[0].items()

dict_items([('id', 53409593), ('usuario', 'Safoan Riyad'), ('resenha_original', "J'aimais bien ChatGPT. Mais la dernière mise à jour a tout gâché. Elle a tout oublié."), ('resenha_pt', 'Eu gosto muito de ChatGPT. A última atualização não funcionou bem. Ela esqueceu tudo.\n'), ('sentimento', 'negativo')])